# Lab SQL 9
In this lab we will find the customers who were active in consecutive months of May and June. Follow the steps to complete the analysis.


## Create a table `rentals_may` to store the data from rental table with information for the month of May.

In [1]:
#rentals_may
#isn't this step redundant and more complex if we want a new table,
#which is essentially just a copy of an already existant table, but then filtered on condition(s)? 
#or did i get these questions wrong?

## Insert values in the table `rentals_may` using the table rental, filtering values only for the month of May.
 

In [2]:
rentals_may = '''
CREATE TABLE rentals_may AS
    SELECT *
    FROM rental
    WHERE month(rental_date) = 5;
    ''' 

## Create a table `rentals_june` to store the data from rental table with information for the month of June.


In [3]:
#rentals_june
#samesies, skipped it

## Insert values in the table `rentals_june` using the table rental, filtering values only for the month of June.
 

In [4]:
rentals_june = '''
CREATE TABLE rentals_june AS
    SELECT *
    FROM rental
    WHERE month(rental_date) = 6;
    ''' 

## Check the number of rentals for each customer for May.


In [5]:
query_may = '''
SELECT customer_id, COUNT(*) AS may 
FROM rentals_may
GROUP BY customer_id;
'''

## Check the number of rentals for each customer for June.
 

In [6]:
query_june = '''
SELECT customer_id, COUNT(*) AS june
FROM rentals_june
GROUP BY customer_id;
'''

## Create a Python connection with SQL database and retrieve the results of the last two queries (also mentioned below) as dataframes:

  - Check the number of rentals for each customer for May
  - Check the number of rentals for each customer for June

    **Hint**: You can store the results from the two queries in two separate dataframes.
 

In [7]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # to get the password without showing the input
password = getpass.getpass()

········


In [8]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'

engine = create_engine(connection_string)

engine.execute("DROP TABLE IF EXISTS rentals_may")
engine.execute("DROP TABLE IF EXISTS rentals_june")
engine.execute(rentals_may)
engine.execute(rentals_june)

In [9]:
data_m = pd.read_sql_query(query_may, engine)
data_m.head()

,customer_id,may
0,130,2
1,459,1
2,408,3
3,333,1
4,222,5


In [10]:
data_j = pd.read_sql_query(query_june, engine)
data_j.head()

,customer_id,june
0,416,5
1,516,6
2,239,5
3,285,3
4,310,6


## Write a function that checks if customer borrowed more or less films in the month of June as compared to May.
 
 **Hint**: For this part, you can create a join between the two dataframes created before, using the merge function available for pandas dataframes. Here is a link to the documentation for the [merge function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html).

In [11]:
data_merged = pd.merge(data_m, data_j, on='customer_id')
data_merged.head()
#i could've also used outer join in this case, no?

,customer_id,may,june
0,130,2,23
1,459,1,7
2,408,3,3
3,333,1,4
4,222,5,2


In [12]:
def compare(x):
    if x['june'] > x['may']:
        return 'more'
    elif x['june'] < x['may']:
        return 'less'
    else:
        return 'same'
data_merged['compare'] = data_merged.apply(compare, axis=1)
data_merged
#how do i make my function more general? aka, how can i replace column specification in the compare function,
#and then apply the function,
#so it would also be applicable to comparing other columns (not the case now, but anyways useful to know)?

,customer_id,may,june,compare
0,130,2,23,more
1,459,1,7,more
2,408,3,3,same
3,333,1,4,more
4,222,5,2,less
...,...,...,...,...
507,431,1,7,more
508,191,2,6,more
509,351,1,3,more
510,10,1,5,more
